# Face Detection and Recognition CNN Models

In [ ]:
import torch
from torch import nn
import scipy.io
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math

# mirar diferencias entre estos dos y elegir uno
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

## Loading the dataset

In [ ]:
class MyData(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        with Image.open(image_path) as image:
            # Apply transformations if specified
            if self.transform:
                image = self.transform(image)
        return image, label

In [ ]:
def load_data(labels_path, labels_wanted='boundaries', tr_size=0.8, val_size=0.1):
    mat = scipy.io.loadmat(labels_path)['AGC_Challenge3_TRAINING'][0]
    data = []
    for entry in mat:
        key = entry[1][0]
        if (labels_wanted == 'boundaries'):
            data.append([key, entry[2]])
        elif (labels_wanted == 'identity'):
            data.append([key, entry[0][0][0]])

    return data

In [ ]:
# transforms son copiados, revisar
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

all_data = load_data('AGC_Challenge3_Training.mat')
train_data, val_test_data = train_test_split(all_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42)


train_dataset = MyData(train_data, transform)
val_dataset = MyData(val_data, transform)
test_dataset = MyData(test_data, transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## The detection model

In [ ]:
class DetectionCNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        # x = self.flatten(x)
        return x

    def fit(self, training_data, loss_fn, optimizer):
        self.train()
        for batch_idx, (image, target) in enumerate(training_data):
            pass


    def evaluate(self):
        self.eval()
        pass

    def predict(self):
        self.eval()
        pass


In [ ]:
detection = DetectionCNNModel().to(device)
print(detection)

In [ ]:
pytorch_total_params = sum(p.numel() for p in detection.parameters())
print(pytorch_total_params)

In [ ]:
torch.save(detection, 'detection_model.pth') # not model.state_dict() because we want to store the class also

## The recognition model

In [ ]:
class RecognitionCNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

In [ ]:
model = RecognitionCNNModel().to(device)
print(model)

In [ ]:
torch.save(model, 'recognition_model.pth') # not model.state_dict() because we want to store the class also